# Python TERMite toolkit - TExpress

We provide a Python library for making calls to our NER engine, TERMite, as well as the TExpress module for defining more complex semantic patterns. The library also enables post-processing of the JSON returned from such requests. This notebook assumes that you're read the example TERMite notebook and walks you through how to make a TExpress call and some of the post-processing of the JSON output.

## Example call to TExpress

The toolkit can also be used to make TExpress calls to identify patterns and extract biomedical relationships. Using TExpress with the toolkit is easy: simply ```import texpress``` from the ```termite_toolkit``` and make a call.

A simple TExpress call is made up of:
* the TERMite API endpoint
* the pattern you wish to search for - this can be created in the TERMite UI
* a TExpress request
* request execution

Below is an example TExpress call with the result being printed to the screen.

In [ ]:
from pprint import pprint
from termite_toolkit import texpress

# specify termite API endpoint
termite_home = "http://localhost:9090/termite"

# specify the pattern you wish to search for- this can created in the TERMite UI
pattern = ":(INDICATION):{0,5}:(GENE)"

t = texpress.TexpressRequestBuilder()

# individually add items to your TERMite request
t.set_url(termite_home)
t.set_text("sildenafil citrate macrophage colony stimulating factor influenza")
t.set_subsume(True)
t.set_input_format("txt")
t.set_output_format("json")
t.set_allow_ambiguous(False)
t.set_pattern(pattern)

# execute the request
texpress_response = t.execute(display_request=False)

pprint(texpress_response, depth=2, compact=True, width=100)

For more information on the TExpress JSON results [click here](https://help.scibite.com/a/solutions/articles/4000021813-anatomy-of-a-texpress-result-server-).

Like TERMite, TExpress calls can be simplified to call options and annotation:


In [ ]:
from pprint import pprint
from termite_toolkit import texpress
import sys
import os

termite_home = "http://localhost:9090/termite"
parentDir = os.path.dirname(os.path.dirname(os.path.abspath("__file__")))  # this line relatively locates the parent directory
input_file = os.path.join(parentDir, 'sample_scripts/medline_sample.zip')
options = {"format": "medline.xml", "output": "json", "pattern": ":(INDICATION):{0,5}:(GENE)",
           "opts"  : "reverse=false"}

texpress_json_response = texpress.annotate_files(termite_home, input_file, options)

# TExpress toolkit library

The standard JSON output isn't very human friendly, so we've added functionality for parsing the JSON and doc.JSONx outputs. The output can be returned as either a dictionary object or as a pandas dataframe.

In [ ]:
pprint(texpress.get_entity_hits_from_json(texpress_response))

In [ ]:
texpress.get_texpress_dataframe(texpress_json_response).head()